In [ ]:
import requests
import os
import json
import pandas as pd
import time
import pymysql

In [ ]:
# Connect to the database
connection = pymysql.connect(host='mentegy-db.****.eu-west-1.rds.amazonaws.com',
                             user='root',
                             password='****',
                             db='mentegy-prod')

In [ ]:
# Creating a cursor
cursor = connection.cursor()

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
%env BEARER_TOKEN= AAAAAAAAAAAAAAAAAAAAAPJZOAEAAAAA4AHhQ5g%2FgMT%2F8%2FSbrwkaATyPFCc%3D6WAGebh9bWk93cLHkDhmfVIVLiGIHqE8sQ8O2Ry7IjihDPVPis 

In [ ]:
def auth():
    return os.environ.get("BEARER_TOKEN")

def create_url():
    return "https://api.twitter.com/2/tweets/sample/stream"

In [ ]:
# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


In [ ]:
def get_rules(headers, bearer_token):
    with requests.get("https://api.twitter.com/2/tweets/search/stream/rules", headers=headers) as response:
        if response.status_code != 200:
            raise Exception(
                "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
            )
    #print(json.dumps(response.json()))
    return response.json()
    

In [ ]:
def delete_all_rules(headers, bearer_token, rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    with requests.post("https://api.twitter.com/2/tweets/search/stream/rules",headers=headers,json=payload) as response:
        if response.status_code != 200:
            raise Exception(
                "Cannot delete rules (HTTP {}): {}".format(
                    response.status_code, response.text
                )
            )
    #print(json.dumps(response.json()))
    

In [ ]:
def set_rules(headers, delete, bearer_token):
    # You can adjust the rules if needed
    sample_rules = [
        {"value": "asian  -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value": "black -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"allah -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"blacks -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"chink -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"chinks -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"dykes -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"faggot -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"faggots -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"fags -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"homo -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"inbred -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"nigger -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"niggers -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"queers -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"raped -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"savages -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"slave -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"spic -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"wetback -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"wetbacks -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"whites -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"fag -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"faggot -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"nigga -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"niggas -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        {"value":"muslims -is:retweet OR from:Ireland OR from:Dublin OR from:Cork OR from:Limerick OR from:Galway"},
        
           
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    
    #print(json.dumps(response.json()))


In [ ]:
def get_stream(headers, set, bearer_token):
    
    r = ''
    while r == '':
        try:
            with requests.get("https://api.twitter.com/2/tweets/search/stream?tweet.fields=created_at&expansions=author_id&user.fields=created_at&place.fields=geo", headers=headers, stream=True,) as response:
            #print(response.status_code)
                if response.status_code != 200:
                    raise Exception(
                        "Cannot get stream (HTTP {}): {}".format(
                            response.status_code, response.text
                        )
                    )
                #iterating over response
                for response_line in response.iter_lines():
                    if response_line:
                        json_response = json.loads(response_line)
                        #converting the created_at key to datetime
                        created_at = pd.to_datetime(json_response['data']['created_at']) #converting to datetime
                        text = json_response['data']['text']
                        ps = analyzer.polarity_scores(text)
                        #forming the insert query
                        sql = "INSERT INTO `app_twitter_streams` (`created_at`, `text`, `compound_score`) VALUES (%s, %s, %s);"
                        #executing the query using the execute function
                        cursor.execute(sql,(created_at,text,ps['compound']))
                        #commit the transaction after inserting every tweet
                        connection.commit()
                        #print(json_response)
        except (requests.exceptions.ConnectionError, requests.exceptions.ChunkedEncodingError) as err:
            #Continue loop after 100 ms
            time.sleep(100)
            continue   

In [ ]:
def main():
    bearer_token = os.environ.get("BEARER_TOKEN")
    headers = create_headers(bearer_token)
    rules = get_rules(headers, bearer_token)
    delete = delete_all_rules(headers, bearer_token, rules)
    set = set_rules(headers, delete, bearer_token)
    get_stream(headers, set, bearer_token)


In [ ]:
if __name__ == "__main__":
    main()
